# Purpose
This notebook exists so that I can familiarise myself with the PyTorch pre-trained model API

In [1]:
from pathlib import Path

from torch.utils.tensorboard import SummaryWriter
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights

from preprocess.loaders import make_hou_loader

In [2]:
# Initialise the model with the latest weights
weights = FCN_ResNet50_Weights.DEFAULT
model = fcn_resnet50(weights=weights)

In [3]:
print(model)

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [4]:
# This pretrained model already has several pixel-wise 
# classification classes.
weights.meta["categories"]

['__background__',
 'aeroplane',
 'bicycle',
 'bird',
 'boat',
 'bottle',
 'bus',
 'car',
 'cat',
 'chair',
 'cow',
 'diningtable',
 'dog',
 'horse',
 'motorbike',
 'person',
 'pottedplant',
 'sheep',
 'sofa',
 'train',
 'tvmonitor']

In [5]:
len(weights.meta["categories"])

21

The model has been trained to predict the classes above. These classes do not appear to be closely related to the classes I am looking to find: `__background__` and `solar_panel`.
The lower-level representations may still be useful for fine-tuning

In [6]:
num_classes = 2

In [7]:
HOU_DIR = Path('data/Hou')
hou_loader = make_hou_loader(HOU_DIR)
model_input = next(iter(hou_loader))
transforms = weights.transforms()

In [8]:
with SummaryWriter('runs/model_exploration') as writer:
    writer.add_graph(
        model,
        input_to_model=transforms(model_input[0]),
        # verbose=True,
        use_strict_trace=False
    )

In [10]:
with SummaryWriter('runs/model_exploration_backbone') as writer:
    writer.add_graph(
        model.backbone,
        input_to_model=transforms(model_input[0]),
        # verbose=True,
        use_strict_trace=False
    )